In [ ]:
import pickle
import numpy as np

In [ ]:
with open('C:/Users/RW565TZ/Practice/train_qa.txt','rb') as f:
    train_data=pickle.load(f)

In [ ]:
with open('C:/Users/RW565TZ/Practice/test_qa.txt','rb') as f:
    test_data=pickle.load(f)

In [ ]:
type(test_data)

In [ ]:
type(train_data)

In [ ]:
len(train_data)

In [ ]:
len(test_data)

In [ ]:
train_data[0]

In [ ]:
' '.join(train_data[0][0])

In [ ]:
' '.join(train_data[0][1])

In [ ]:
train_data[0][2]

In [ ]:
all_data=test_data +train_data

In [ ]:
len(all_data)

In [ ]:
vocab=set()

for story,question,answer in all_data:
    vocab =vocab.union(set(story))
    vocab =vocab.union(set(question))

In [ ]:
vocab.add('no')

In [ ]:
vocab.add('yes')

In [ ]:
vocab

In [ ]:
vocab_len = len(vocab)+1

In [ ]:
vocab_len

In [ ]:
# Longest story

all_story_lens = [len(data[0]) for data in all_data]

In [ ]:
max(all_story_lens)

In [ ]:
max_story_len = max(all_story_lens)

In [ ]:
max_question_len = max([len(data[1]) for data in all_data])

In [ ]:
max_question_len

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [ ]:
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [ ]:
tokenizer.word_index

In [ ]:
train_story_text=[]
train_question_text=[]
train_answer_text=[]

In [ ]:
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answer_text.append(answer)

In [ ]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [ ]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    # Story X
    X=[]
    # Question Xq
    Xq=[]
    # Answer y
    Y=[]
    
    for story,query,answer in data:
        # Create list for story
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index)+1)
        
        y[word_index[answer]]=1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))

In [ ]:
inputs_train,queries_train,answers_train = vectorize_stories(train_data)

In [ ]:
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

In [ ]:
tokenizer.word_index['yes']

In [ ]:
tokenizer.word_index['no']

In [ ]:
sum(answers_test)

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate

In [ ]:
input_seq = Input((max_story_len,))
question = Input((max_question_len,))

In [ ]:
vocab_size =len(vocab)+1

In [ ]:
# Input encoder m

input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [ ]:
input_encoder_m

In [ ]:
# Question input encoder c

input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

In [ ]:
input_encoder_c

In [ ]:
# Question encoder
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

In [ ]:
input_encoded_m = input_encoder_m(input_seq)
input_encoded_c = input_encoder_c(input_seq)
question_encoded =question_encoder(question)

In [ ]:
match = dot([input_encoded_m,question_encoded],axes=(2,2))
match=Activation('softmax')(match)

In [ ]:
response = add([match,input_encoded_c])
response=Permute((2,1))(response)

In [ ]:
answer = concatenate([response,question_encoded])

In [ ]:
answer

In [ ]:
from tensorflow.keras.layers import LSTM
answer = LSTM(32)(answer)

In [ ]:
answer=Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)

In [ ]:
answer = Activation('softmax')(answer)
model = Model([input_seq,question],answer)

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=50,validation_data=([inputs_test, queries_test], answers_test))


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
pred_results = model.predict(([inputs_test,queries_test]))

val_max = np.argmax(pred_results[0])

In [ ]:
for key,val in tokenizer.word_index.items():
    if val==val_max:
        k = key

In [ ]:
k

In [ ]:
pred_results[0][val_max]

In [ ]:
my_story = 'John left the kitchen. Sandra dropped the football in the garden'

In [ ]:
my_story.split()

In [ ]:
my_question = ' Is the football in the garden'
my_question.split()

In [ ]:
mydata = ([my_story.split(),my_question.split(),'yes'])

In [ ]:
mydata

In [ ]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [ ]:
pred_results = model.predict(([my_story,my_ques]))

val_max = np.argmax(pred_results[0])

In [ ]:
for key,val in tokenizer.word_index.items():
    if val==val_max:
        k = key

In [ ]:
k

In [ ]:
pred_results[0][val_max]